# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [5]:
# Your code here
import sqlite3
import pandas as pd

conn = sqlite3.Connection('data.sqlite')
cur = conn.cursor()



## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [27]:
# Your code here
# Your code here
cur.execute("""SELECT lastname, firstname, o.city
                      FROM employees e
                      JOIN offices o
                      USING(officeCode)
                      WHERE o.city == 'Boston'
                      ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()


,lastName,firstName,city
0,Firrelli,Julie,Boston
1,Patterson,Steve,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [64]:
# Your code here
# cur.execute("""SELECT count(lastName) as Num_Emps, o.city
#                       FROM employees e
#                       RIGHT JOIN offices o
#                       USING(officeCode)
#                       GROUP BY officeCode
#                       --HAVING Num_Emps == 0
#                       ;""")

cur.execute("""
            SELECT officeCode, count(lastName)
            FROM offices o
            JOIN employees e
            
            USING(officeCode)
            GROUP BY officeCode
            
            ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(df.tail(10))


cur.execute("""SELECT *
                      FROM offices
                      ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(df.tail(10))

try:
    cur.execute("""SELECT *
          FROM employees
          WHERE officeCode == '27'
          ;""")

    df = pd.DataFrame(cur.fetchall())
    df.columns = [x[0] for x in cur.description]
    print(df.tail(10))
except:
    print('no records located. Office 27 has no employees')

   officeCode  count(lastName)
0           1                6
1           2                2
2           3                2
3           4                5
4           5                2
5           6                4
6           7                2
   officeCode           city             phone              addressLine1  \
0           1  San Francisco   +1 650 219 4782         100 Market Street   
1           2         Boston   +1 215 837 0825          1550 Court Place   
2           3            NYC   +1 212 555 3000      523 East 53rd Street   
3           4          Paris   +33 14 723 4404  43 Rue Jouffroy D'abbans   
4           5          Tokyo   +81 33 224 5000               4-1 Kioicho   
5           6         Sydney   +61 2 9264 2451     5-11 Wentworth Avenue   
6           7         London  +44 20 7877 2041       25 Old Broad Street   
7          27         Boston   +1 977 299 8345      105 Cambridge Street   

  addressLine2       state    country postalCode territory  
0    S

## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold

In [82]:
# Your code here
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
cur.execute("""
            SELECT /*max(textDescription),*/ max(htmlDescription) as HTML_Desc, /*productName,*/ lastName, firstName, orderNumber
            FROM orders o
            JOIN customers c
            USING(customerNumber)
            JOIN employees e
            ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN orderdetails 
            USING(orderNumber)
            JOIN products
            USING(productCode)
            JOIN productlines
            USING(productLine)
            GROUP BY lastName, firstName, orderNumber
            
            ;""")
#cur.execute("""Select htmlDescription from productlines""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(df.tail(15))

    HTML_Desc lastName firstName  orderNumber
311             Vanauf    George        10233
312             Vanauf    George        10235
313             Vanauf    George        10236
314             Vanauf    George        10248
315             Vanauf    George        10251
316             Vanauf    George        10263
317             Vanauf    George        10283
318             Vanauf    George        10292
319             Vanauf    George        10308
320             Vanauf    George        10313
321             Vanauf    George        10329
322             Vanauf    George        10331
323             Vanauf    George        10353
324             Vanauf    George        10401
325             Vanauf    George        10413


In [85]:
# Your code here
#find top sums of credit limits of customers for each employee

cur.execute("""
            SELECT lastName as EmpLastName, sum(creditLimit) as sum_Of_Credit
            FROM employees e
            JOIN customers c
            ON e.employeeNumber = c.salesRepEmployeeNumber
            GROUP BY lastName, firstName
            ORDER BY sum_Of_Credit
            
            ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(df.tail(15))

   EmpLastName  sum_Of_Credit
0     Thompson         391600
1        Nishi         419500
2       Fixter         430300
3       Gerard         431500
4     Firrelli         443500
5        Marsh         460400
6        Tseng         466300
7    Patterson         489200
8       Bondur         517400
9     Jennings         602600
10   Hernandez         642500
11      Vanauf         647100
12        Bott         729500
13       Jones         769600
14    Castillo         813400


In [88]:
# Your code here
# Your code here
#Find total Payment Amounts for each Employee

cur.execute("""
            SELECT lastName as EmpLastName, firstName as EmpFirstName, sum(amount) as sum_Of_Pmt_Amt
            FROM employees e
            JOIN customers c
            ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN payments p
            USING(customerNumber)
            GROUP BY lastName, firstName
            ORDER BY sum_Of_Pmt_Amt DESC
            
            ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(df.tail(15))

   EmpLastName EmpFirstName  sum_Of_Pmt_Amt
0    Hernandez       Gerard      1112003.81
1     Jennings       Leslie       989906.55
2     Castillo       Pamela       750201.87
3         Bott        Larry       686653.25
4        Jones        Barry       637672.65
5       Vanauf       George       584406.80
6       Bondur         Loui       569485.75
7       Fixter         Andy       509385.82
8        Marsh        Peter       497907.16
9        Tseng     Foon Yue       488212.67
10       Nishi         Mami       457110.07
11   Patterson        Steve       449219.13
12      Gerard       Martin       387477.47
13    Firrelli        Julie       386663.20
14    Thompson       Leslie       347533.03


## Level Up: Display the names of every individual product that each employee has sold

In [92]:
# Your code here
cur.execute("""
            SELECT max(productName) as ProdName, lastName, firstName/*, orderNumber*/
            FROM orders o
            JOIN customers c
            USING(customerNumber)
            JOIN employees e
            ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN orderdetails 
            USING(orderNumber)
            JOIN products
            USING(productCode)
            GROUP BY lastName, firstName, orderNumber
            
            ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(df.tail(15))

                                        ProdName lastName firstName
311                    The USS Constitution Ship   Vanauf    George
312                               P-51-D Mustang   Vanauf    George
313                        1996 Moto Guzzi 1100i   Vanauf    George
314                    The USS Constitution Ship   Vanauf    George
315         2003 Harley-Davidson Eagle Drag Bike   Vanauf    George
316                               P-51-D Mustang   Vanauf    George
317                    The USS Constitution Ship   Vanauf    George
318             Diamond T620 Semi-Skirted Tanker   Vanauf    George
319                               P-51-D Mustang   Vanauf    George
320  1996 Peterbilt 379 Stake Bed with Outrigger   Vanauf    George
321                             Boeing X-32A JSF   Vanauf    George
322                           2002 Yamaha YZR M1   Vanauf    George
323                               P-51-D Mustang   Vanauf    George
324                               P-51-D Mustang

## Level Up: Display the Number of Products each employee has sold

In [95]:
# Your code here

cur.execute("""
            SELECT count(productName) as NumOfProducts, lastName, firstName/*, orderNumber*/
            FROM orders o
            JOIN customers c
            USING(customerNumber)
            JOIN employees e
            ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN orderdetails 
            USING(orderNumber)
            JOIN products
            USING(productCode)
            GROUP BY lastName, firstName
            ORDER BY NumOfProducts DESC
            
            ;""")
#cur.execute("""Select htmlDescription from productlines""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(df.tail(15))

    NumOfProducts   lastName firstName
0             396  Hernandez    Gerard
1             331   Jennings    Leslie
2             272   Castillo    Pamela
3             236       Bott     Larry
4             220      Jones     Barry
5             211     Vanauf    George
6             185     Fixter      Andy
7             185      Marsh     Peter
8             177     Bondur      Loui
9             152  Patterson     Steve
10            142      Tseng  Foon Yue
11            137      Nishi      Mami
12            124   Firrelli     Julie
13            114     Gerard    Martin
14            114   Thompson    Leslie


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!